In [31]:
# necessary Modules

import random
from hashlib import sha256, sha3_256
import sympy

In [32]:
# function to create s (secret key) and v (public key) with given g (generator) and p (prime mod)  
def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v


# function to generate n random secret and public keys
def generate_n_random_keys(n):
    # a list to store all the pubkey and secrect keys
    keys_list = []
    for _ in range(n):
        # generate random keys
        x_temp, y_temp = generate_keys(g, p)
        # append random keys
        keys_list.append([x_temp, y_temp])
    return keys_list

# function to concat the real key to the random key list and shuffle
def concat_and_shuffle_total_keys(keys_list, x, y):
    # append the real key
    keys_list.append([x, y])
    # do the shuffle
    random.shuffle(keys_list)
    return keys_list

# function to get the real public key's corresponding position
def get_real_key_position(keys_list, y):
    # make the public keys into one list for index
    public_key_list = [key[1] for key in keys_list]
    # return the real public key index
    return public_key_list.index(y)

# function to get the Y, which is the string containing all the public key used in this signature
def get_Y(keys_list):
    # make the public keys into one list for getting the Y
    public_key_list = [key[1] for key in keys_list]
    # generate Y, which is the string containing all the public key used in this signature
    Y = ''.join([str(i) for i in public_key_list])
    return Y

# function to get the initial random value r and R used in the signature
def get_initial_random_r_and_R(g, p):
    r, R = generate_keys(g, p)
    return r, R

# function to generate the signature
def sign(msg, Y, g, r, p, keys_list, x_real, y_real):
    # find the real pubkey's position and +1 to get the next position
    idx = (get_real_key_position(total_keys_list, y_real) + 1) % len(total_keys_list)
    # initial the e_list with None (which represents c in the thesis)
    e_list = [None for _ in range(len(keys_list))]
    # initial the signature list with None 
    sig_list = [None for _ in range(len(keys_list))]
    # calculate the h with Y and using another hash function (thesis 4.1.1)
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)
    # calculate y wave (thesis 4.1.1)
    y_wave = pow(h, x_real, p)
    # calculate the first e with the random number "r" (thesis 4.1.2)
    e_initial = int(sha256((str(Y) + str(y_wave) +str(msg) + str(pow(g,r,p)) + str(pow(h,r,p))).encode()).hexdigest(), 16)
    # place the "e" into e_list
    e_list[idx] = e_initial
    # do the while loop, in order to calculate each position's value until the real public key's position
    while keys_list[idx][1] != y_real:
        # find the "e" used in last term loop
        e_last_one = e_list[idx]
        # generate the random value for the signature (thesis 4.1.3 start)
        _, random_sig = generate_keys(g, p)
        # place it to the position
        sig_list[idx] = random_sig
        # find this term cooresponding y
        temp_y = keys_list[idx][1]
        # do the calculation (thesis 4.1.3 for last two parts)
        # gy part
        calculation = (pow(g, random_sig, p)*pow(temp_y, e_last_one, p)) % p
        # hy part
        calculation_1 = (pow(h, random_sig, p)*pow(y_wave, e_last_one, p)) % p
        # calculate the e for this term (thesis 4.1.3 end)
        temp_e = int(sha256((str(Y) + str(y_wave) +str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
        # index += 1
        idx = (idx + 1) % len(total_keys_list)
        # place the "e"
        e_list[idx] = temp_e
        
    # q, see paper page 6, part 4, the first sentence. 
    # q = (p - 1)/2
    q = (p-1)//2
    print('q:', q)
    # do the real signature (thesis 4.1.4)
    s_real = (r - x_real * e_list[idx]) % q
    sig_list[idx] = s_real
    public_key_list = [key[1] for key in keys_list] 
    total_res = {
        'e1': e_list[0],
        'sig_list': sig_list,
        'public_key_list': public_key_list,
        'tag': y_wave,
        'msg': msg,
    }
    return total_res
    
# generate the safe prime
def generate_safe_prime(bits):
    while True:
        p = sympy.randprime(2**(bits-1), 2**bits)
        if sympy.isprime((p-1)//2):
            return int(p)

# define the safe prime p in 128 bits, and the g (primitive root of p) used in the whole process
p = generate_safe_prime(128)
print('prime: ',p)
g = 5 
msg = 'Hello, Ring Signature'
random_keys_count = 10

# main function
random_keys_list = generate_n_random_keys(random_keys_count)
x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, p)
total_res = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res)

prime:  224710685719495060258256606457699928103
q: 112355342859747530129128303228849964051


{'e1': 57289108914561308583482816483541787373368309869154547389554140862688771092189,
 'sig_list': [185014343212421197658956846310602882203,
  87905042311524951582304053054954409932,
  1407068846186674425068020270838055845,
  59494978662547562864270762870404748077,
  69122631859187469496024365574833293862,
  43268847499326367393398359892719938586,
  12313177626908770388208808557904716240,
  119089755959182193078468326357026353733,
  73840675200910473813474717286974005118,
  82766585703582382863983451782946278947,
  75791769157466589373832606580740826208],
 'public_key_list': [40328962056998690303794782529429710469,
  159523871802407578067319899303667160141,
  139192738075522857220810422713151095671,
  140756302582080806639952776904337860908,
  102858608247867137114561849586024472391,
  147008866697392189886789258556433265546,
  176461958112014665175457513823256941230,
  32366713811293173201088693378151522014,
  65772743918354511489856164815128389044,
  728744354293382165060741446349236

In [34]:
def verify(total_res):
    public_key_list = total_res['public_key_list']
    Y = ''.join([str(i) for i in public_key_list])
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)

    sig_list = total_res['sig_list']
    temp_e_V = e1 = total_res['e1']
    msg = total_res['msg']
    tag = total_res['tag']
    
    for i in range(len(sig_list)):
        print('temp_e_V:',  temp_e_V)
        temp_sig = sig_list[i]
        temp_y = public_key_list[i]
        calculation = (pow(g, temp_sig, p)*pow(temp_y, temp_e_V, p)) % p
        calculation_1 = (pow(h, temp_sig, p)*pow(tag, temp_e_V, p)) % p
        temp_e_V = int(sha256((str(Y) + str(tag) + str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
    
    print(e1)
    print(temp_e_V)
    print(temp_e_V == e1) 

verify(total_res)

temp_e_V: 57289108914561308583482816483541787373368309869154547389554140862688771092189
temp_e_V: 64884885644900410094068242260614111778026812039169265266119921306870236563483
temp_e_V: 84885438583525891615065393508968103827969350553851062133250424969298778915682
temp_e_V: 15588086540735453430203215339652242816047579735841822014556550421205205680317
temp_e_V: 59774800618099216636571683466594330277393769294114073282598289819181575924185
temp_e_V: 51668408493385968953219341330029335064564123164873920192177640664009722583593
temp_e_V: 23541980456627074391408796565910213729698595877263671415356192567442911943252
temp_e_V: 35192646773829814988852719048334599990838152158104604197095440555829395345003
temp_e_V: 780812297941193985852506879121353809767004499491965843909606232764403231269
temp_e_V: 20946136465378422776343977428050939472013183682949073899286411148475088098411
temp_e_V: 60189088957649743522471094953355502383429091935995091159071400713068966653515
5728910891456130858348281648354178

In [35]:
def check_tag(signature_res, signature_res_1):
    return signature_res['tag'] == signature_res_1['tag']

msg_1 = 'new test'
total_res_1 = sign(msg_1, Y, g, r, p, total_keys_list, x_real, y_real)
display(total_res_1)
print("total_res_1['tag'] == total_res['tag']:",check_tag(total_res, total_res_1))

q: 112355342859747530129128303228849964051


{'e1': 92865454267432715384169708901225793628057433129744614178406322839189007592096,
 'sig_list': [31079017130716592821175011228261016698,
  116707165795621439822632451143729802746,
  206747499993751972925314312024174295260,
  172419247805942981137156052491505808590,
  5408127746634669863457803862430107325,
  218149802009101260757778865081820174004,
  116549009289122950022599272023485496372,
  136862275923730567302755815059633770022,
  22171831460143077040313974754346817740,
  69861416519405826021093069159671102293,
  197221975559327306241267212732644268638],
 'public_key_list': [40328962056998690303794782529429710469,
  159523871802407578067319899303667160141,
  139192738075522857220810422713151095671,
  140756302582080806639952776904337860908,
  102858608247867137114561849586024472391,
  147008866697392189886789258556433265546,
  176461958112014665175457513823256941230,
  32366713811293173201088693378151522014,
  65772743918354511489856164815128389044,
  7287443542933821650607414463

total_res_1['tag'] == total_res['tag']: True


In [36]:
def is_primitive_root(p, g):
    n = p - 1
    factors = []
    i = 2
    while i * i <= n:
        if n % i == 0:
            factors.append(i)
            while n % i == 0:
                n //= i
        i += 1
    if n > 1:
        factors.append(n)

    for factor in factors:
        if pow(g, (p-1) // factor, p) == 1:
            return False
    return True

p = 1000000007 
g = 5 

result = is_primitive_root(p, g)
print(f"g = {g} is a primitive root of p = {p}: {result}")


g = 5 is a primitive root of p = 1000000007: True


In [37]:
# define the prime p, and the g (primitive root of p) used in the whole process
p = 1000000007  # prime
g = 5  # primitive root of p

msg = 'Hello, Ring Signature'
random_keys_count = 10

random_keys_list = generate_n_random_keys(random_keys_count)
# x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, p)
total_res_2 = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res_2)
print("total_res['tag'] == total_res_2['tag']:",check_tag(total_res, total_res_2))

q: 500000003


{'e1': 52932635015520690102669606878439076374223921588851612617365947714208443184858,
 'sig_list': [69816858,
  281124807,
  316353968,
  400840751,
  802562457,
  5004604,
  920553542,
  100736813,
  340951710,
  188267126,
  456394787],
 'public_key_list': [602558824,
  544878822,
  824453574,
  438080607,
  703854269,
  573567152,
  637907361,
  842998597,
  735999944,
  769438731,
  65772743918354511489856164815128389044],
 'tag': 366852169,
 'msg': 'Hello, Ring Signature'}

total_res['tag'] == total_res_2['tag']: False
